<a href="https://colab.research.google.com/github/salav2006poudyal2025/5CS037_worksheet_sulavpoudyal/blob/main/5CS037_worksheet8_sulavpoudyal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
class CustomDecisionTree:
  def __init__(self, max_depth=None):
    """
    Initializes the decision tree with the specified maximum depth.
    Parameters:
    max_depth (int, optional): The maximum depth of the tree. If None, the tree is expanded until all
    leaves are pure or contain fewer than the minimum samples required to split.
    """
    self.max_depth = max_depth
    self.tree = None
  def fit(self, X, y):
    """
    Trains the decision tree model using the provided training data.
    Parameters:
    X (array-like): Feature matrix (n_samples, n_features) for training the model.
    y (array-like): Target labels (n_samples,) for training the model.
    """
    self.tree = self._build_tree(X, y)
  def _build_tree(self, X, y, depth=0):
    """
    Recursively builds the decision tree by splitting the data based on the best feature and threshold
    .
    Parameters:
    X (array-like): Feature matrix (n_samples, n_features) for splitting.
    y (array-like): Target labels (n_samples,) for splitting.
    depth (int, optional): Current depth of the tree during recursion.
    Returns:
    dict: A dictionary representing the structure of the tree, containing the best feature index,
    threshold, and recursive tree nodes.
    """
    num_samples, num_features = X.shape
    unique_classes = np.unique(y)
    # Stopping conditions: pure node or reached max depth
    if len(unique_classes) == 1:
      return {'class': unique_classes[0]}
    if num_samples == 0 or (self.max_depth and depth >= self.max_depth):
      return {'class': np.bincount(y).argmax()}
    # Find the best split based on Information Gain (using Entropy)
    best_info_gain = -float('inf')
    best_split = None
    for feature_idx in range(num_features):
      thresholds = np.unique(X[:, feature_idx])
      for threshold in thresholds:
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask
        left_y = y[left_mask]
        right_y = y[right_mask]
        info_gain = self._information_gain(y, left_y, right_y)
        if info_gain > best_info_gain:
          best_info_gain = info_gain
          best_split = {
            'feature_idx': feature_idx,
            'threshold': threshold,
            'left_y': left_y,
            'right_y': right_y,
          }

    if best_split is None:
      return {'class': np.bincount(y).argmax()}
    # Recursively build the left and right subtrees
    left_tree = self._build_tree(X[best_split['left_y']], best_split['left_y'], depth + 1)
    right_tree = self._build_tree(X[best_split['right_y']], best_split['right_y'], depth + 1)

    return {'feature_idx': best_split['feature_idx'], 'threshold': best_split['threshold'],
          'left_tree': left_tree, 'right_tree': right_tree}
  def _information_gain(self, parent, left, right):
    """
    Computes the Information Gain between the parent node and the left/right child nodes.
    Parameters:
    parent (array-like): The labels of the parent node.
    left (array-like): The labels of the left child node.
    right (array-like): The labels of the right child node.
    Returns:
    float: The Information Gain of the split.
    """
    parent_entropy = self._entropy(parent)
    left_entropy = self._entropy(left)
    right_entropy = self._entropy(right)
    # Information Gain = Entropy(parent) - (weighted average of left and right entropies)
    weighted_avg_entropy = (len(left) / len(parent)) * left_entropy + (len(right) / len(parent))*right_entropy
    return parent_entropy - weighted_avg_entropy
  def _entropy(self, y):
    """
    Computes the entropy of a set of labels.
    Parameters:
    y (array-like): The labels for which entropy is calculated.
    Returns:
    float: The entropy of the labels.
    """
    # Calculate the probability of each class
    class_probs = np.bincount(y) / len(y)
    # Compute the entropy using the formula: -sum(p * log2(p))
    return -np.sum(class_probs * np.log2(class_probs + 1e-9)) # Added small epsilon to avoid log(0)
  def predict(self, X):
    """
    Predicts the target labels for the given test data based on the trained decision tree.
    Parameters:
    X (array-like): Feature matrix (n_samples, n_features) for prediction.
    Returns:
    list: A list of predicted target labels (n_samples,).
    """
    return [self._predict_single(x, self.tree) for x in X]
  def _predict_single(self, x, tree):
    """
    Recursively predicts the target label for a single sample by traversing the tree.
    Parameters:
    x (array-like): A single feature vector for prediction.
    tree (dict): The current subtree or node to evaluate.
    Returns:
    int: The predicted class label for the sample.
    """
    if 'class' in tree:
      return tree['class']
    feature_val = x[tree['feature_idx']]
    if feature_val <= tree['threshold']:
      return self._predict_single(x, tree['left_tree'])
    else:
      return self._predict_single(x, tree['right_tree'])

In [ ]:

# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target
# Split into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_custom = custom_tree.predict(X_test)
# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")

Custom Decision Tree Accuracy: 0.8000


In [ ]:
# Train the Scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_sklearn = sklearn_tree.predict(X_test)
# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")

Scikit-learn Decision Tree Accuracy: 1.0000


In [ ]:
print(f"Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")

Accuracy Comparison:
Custom Decision Tree: 0.8000
Scikit-learn Decision Tree: 1.0000


In [ ]:
# Import necessary libraries
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import f1_score, mean_squared_error
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load Wine dataset
wine_data = load_wine()
X_wine = wine_data.data
y_wine = wine_data.target

# Split the dataset into training and testing sets
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(
    X_wine, y_wine, test_size=0.2, random_state=42
)

# Initialize classifiers
dt_classifier = DecisionTreeClassifier(random_state=42)
rf_classifier = RandomForestClassifier(random_state=42)

# Train the models
dt_classifier.fit(X_train_w, y_train_w)
rf_classifier.fit(X_train_w, y_train_w)

# Make predictions
y_pred_dt = dt_classifier.predict(X_test_w)
y_pred_rf = rf_classifier.predict(X_test_w)

# Calculate F1 scores
f1_dt = f1_score(y_test_w, y_pred_dt, average='macro')
f1_rf = f1_score(y_test_w, y_pred_rf, average='macro')

# Print results
print("=== Classification Results ===")
print(f"Decision Tree F1 Score: {f1_dt:.4f}")
print(f"Random Forest F1 Score: {f1_rf:.4f}")

print("\n=== F1 Score Comparison ===")
print("Decision Tree:", f1_dt)
print("Random Forest:", f1_rf)

=== Classification Results ===
Decision Tree F1 Score: 0.9425
Random Forest F1 Score: 1.0000

=== F1 Score Comparison ===
Decision Tree: 0.9424740010946907
Random Forest: 1.0


In [ ]:
# Hyperparameter grid for Random Forest Classifier
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)

grid_search.fit(X_train_w, y_train_w)

print("\n=== GridSearchCV Results ===")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation F1 Score: {grid_search.best_score_:.4f}")

best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test_w)
f1_best = f1_score(y_test_w, y_pred_best, average='weighted')
print(f"Test Set F1 Score with Best Model: {f1_best:.4f}")


=== GridSearchCV Results ===
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation F1 Score: 0.9783
Test Set F1 Score with Best Model: 1.0000


In [ ]:

# Load regression dataset
from sklearn.datasets import load_diabetes
# Load Diabetes dataset (Regression)
diabetes = load_diabetes()
X_reg = diabetes.data
y_reg = diabetes.target

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

dt_regressor = DecisionTreeRegressor(random_state=42)
rf_regressor = RandomForestRegressor(random_state=42)

dt_regressor.fit(X_train_r, y_train_r)
rf_regressor.fit(X_train_r, y_train_r)

y_pred_dt_r = dt_regressor.predict(X_test_r)
y_pred_rf_r = rf_regressor.predict(X_test_r)

mse_dt = mean_squared_error(y_test_r, y_pred_dt_r)
mse_rf = mean_squared_error(y_test_r, y_pred_rf_r)

print("\n=== Regression Results ===")
print(f"Decision Tree Regressor MSE: {mse_dt:.4f}")
print(f"Random Forest Regressor MSE: {mse_rf:.4f}")


=== Regression Results ===
Decision Tree Regressor MSE: 4976.7978
Random Forest Regressor MSE: 2952.0106


In [ ]:
# Hyperparameter tuning for Random Forest Regressor
param_dist_rf_reg = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist_rf_reg,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_r, y_train_r)

print("\n=== RandomizedSearchCV Results ===")
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best Cross-Validation MSE: {-random_search.best_score_:.4f}")

best_rf_reg = random_search.best_estimator_
y_pred_best_reg = best_rf_reg.predict(X_test_r)
mse_best = mean_squared_error(y_test_r, y_pred_best_reg)
print(f"Test Set MSE with Best Model: {mse_best:.4f}")


=== RandomizedSearchCV Results ===
Best Parameters: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10}
Best Cross-Validation MSE: 3318.2050
Test Set MSE with Best Model: 2828.4947
